In [ ]:
!pip install esupar transformers triton
models=[
  "KoichiYasuoka/modernbert-small-classical-chinese-ud-square",
  "KoichiYasuoka/modernbert-base-classical-chinese-ud-square",
  "KoichiYasuoka/modernbert-large-classical-chinese-ud-square",
  "esupar"
]
import os,sys,subprocess
url="https://github.com/UniversalDependencies/UD_Classical_Chinese-Kyoto"
d=os.path.basename(url)
!test -d {d} || git clone --depth=1 {url}
!for F in train dev test ; do cp {d}/*-$$F.conllu $$F.conllu ; done
url="https://universaldependencies.org/conll18/conll18_ud_eval.py"
c=os.path.basename(url)
!test -f {c} || curl -LO {url}
with open("test.conllu","r",encoding="utf-8") as r:
  s=[t[8:].strip() for t in r if t.startswith("# text =")]
for mdl in models:
  if mdl=="esupar":
    import esupar
    nlp=esupar.load("lzh")
  else:
    from transformers import pipeline
    nlp=pipeline("universal-dependencies",mdl,trust_remote_code=True)
  for f in ["dev.conllu","test.conllu"]:
    with open(f,"r",encoding="utf-8") as r:
      s=[t[8:].strip() for t in r if t.startswith("# text =")]
    with open("result-"+f,"w",encoding="utf-8") as w:
      for t in s:
        if model=="esupar":
          print(nlp(t),file=w)
        else:
          try:
            w.write(nlp(t,aggregation_strategy="simple"))
          except:
            break
  os.system(f"mkdir -p result/{mdl}")
  with open(f"result/{mdl}/result.txt","w",encoding="utf-8") as w:
    for f in ["dev.conllu","test.conllu"]:
      p=subprocess.run([sys.executable,c,"-v",f,"result-"+f],
        encoding="utf-8",stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
      print(f"\n*** {mdl} ({f})",p.stdout,sep="\n",file=w)
!( cd result && cat `find {" ".join(models)} -name result.txt` )